In [155]:
import os
import glob
import imghdr
import openpyxl
import cv2
from PIL import Image

In [40]:

# 定数設定
INPUT_IMG_DIR = '.\\images\\' # 貼り付ける画像を置いておくルートディレクトリ
SHEET_TITLE = '画像貼り付け' # シート名の設定
RESULT_FILE_NAME = '.\\result.xlsx' # 結果を保存するファイル名

# 変数
max_height = ['3'] # 各行の画像の高さの最大値を保持

In [138]:

'''
１．列番号から 1 引いた数を 26 で割り、結果の整数値を変数 i に代入します。

２．列番号から i に 26 を掛けた数を引き、結果の整数値を変数 j に代入します。

３．整数値を対応する英文字に変換します。i および j は、それぞれ 0 ～ 9 および 1 ～ 26 の値になります。
'''
def toAlpha2(num):
    i = int((num-1)/26)
    j = int(num-(i*26))
    Alpha = ''
    for z in i,j:
        if z != 0:
            Alpha += chr(z+64)
    return Alpha

In [41]:
def get_file_names(set_dir_name):
    """
    ディレクトリ内のファイル名取得（ファイル名のみの一覧を取得）
    """
    file_names = os.listdir(set_dir_name)
    temp_full_file_names = [os.path.join(set_dir_name, file_name) for file_name in file_names if os.path.isfile(os.path.join(set_dir_name, file_name))] # ファイルかどうかを判定
    return temp_full_file_names

In [164]:
def attach_img(target_full_file_names, set_column_idx, set_dir_name):
    """
    画像を呼び出して、Excelに貼り付け
    """
    set_row_idx = 1
    column_letter = toAlpha2(ws.cell(row=set_row_idx, column=set_column_idx).column) # セルの行列番号から、そのセルの列番号の文字列を取得
    print('column letter:',column_letter)
    ws.cell(row=1, column=set_column_idx).value = set_dir_name # 各列の1行目に、貼り付ける画像があるディレクトリ名を入力
    max_width = 0 # 画像の幅の最大値を保持するための変数
    target_full_file_names.sort() # ファイル名でソート
    for target_file in target_full_file_names:
        if imghdr.what(target_file) != None: # 画像ファイルかどうかの判定
            
            img = openpyxl.drawing.image.Image(target_file)
            print('[' + str(column_letter) + '][' + str(set_row_idx) + ']' + target_file + 'を貼り付け')
 
            # 画像のサイズを取得して、セルの大きさを合わせる（画像同士が重ならないようにするため）
            size_img = cv2.imread(target_file)
            height, width = size_img.shape[:2]
            if max_width < width:
                max_width = width
            if not max_height[set_row_idx-1:set_row_idx]: # 配列「max_height」において、「set_row_idx」番目の要素が存在しなければ、挿入
                max_height.insert(set_row_idx-1, height)
            if int(max_height[set_row_idx-1]) < height:
                max_height[set_row_idx-1] = str(height)
            ws.row_dimensions[set_row_idx+1].height = int(max_height[set_row_idx-1]) * 0.75
            ws.column_dimensions[str(column_letter)].width = int(max_width) * 0.13
            cell_address = ws.cell(row=set_row_idx, column=set_column_idx).coordinate # セルの行列番号から、そのセルの番地を取得
            img.anchor = cell_address
            ws.add_image(img) # シートに画像貼り付け
 
        set_row_idx += 1

In [165]:
# ワークブック設定
wb = openpyxl.Workbook()
ws = wb.worksheets[0] # 1番目のシートを編集対象にする
ws.title = SHEET_TITLE # 1番目のシートに名前を設定

# 貼り付ける画像を置いておくルートディレクトリ内のディレクトリ名を再帰的に取得
dirs = glob.glob(os.path.join(INPUT_IMG_DIR, '**' + os.sep), recursive=True)

column_idx = 1
# 各ディレクトリについて操作
for dir_name in dirs:
    f_names = get_file_names(dir_name) # ファイル名取得
    attach_img(f_names, column_idx, dir_name) # 画像貼り付け設定
    column_idx += 1 # 次の列へ・・・

# ファイルへの書き込み
wb.save(RESULT_FILE_NAME)

column letter: A
column letter: B
[B][1].\images\bing_images\kouro.pngを貼り付け
[B][2].\images\bing_images\screen.pngを貼り付け
column letter: C


In [4]:
#Excelファイル読込
workbook = openpyxl.load_workbook(name)
sheet = workbook.active

#画像を選択
img_to_excel = openpyxl.drawing.image.Image(img_dir)

#指定の位置に画像を添付
sheet.add_image(img_to_excel, 'B3')

#保存
workbook.save(name)